In [ ]:
import numpy as np
import tensorflow as tf

from utils.loadData import loadData
from lstm.preprocessing import normalize, convertToClassification, normalizeAtOnce
from lstm.model import getModel
from lstm.visualization import show_result, visualize_loss, visualize_accuracy

## Load the data

Already downloaded using src/utils/fetchStockData

In [ ]:
try:  
  df = loadData()
except:
  print("Please, download BTC data. See README.")

## Preprocessing

Normalisation, train/test split and preparing for LSTM processing

In [ ]:
features = np.array(df["Close"]).reshape(-1, 1)
normalized_data = normalize(features)

In [ ]:
split_fraction = 0.715
train_split = int(split_fraction * int(df.shape[0]))

past = 100
future = 14
batch_size = 128
epochs = 20

train_data = normalized_data[:train_split]
val_data = normalized_data[train_split:]

In [ ]:
start = past + future
end = start + train_split

x_train = train_data
y_train = normalized_data[start:end]

dataset_train = tf.keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=past,
    batch_size=batch_size,
)

x_end = len(val_data) - past - future
label_start = train_split + past + future

x_val = val_data
y_val = normalized_data[label_start:]
y_val = convertToClassification(x_val, y_val, past+future)

dataset_val = tf.keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=past,
    batch_size=batch_size,
)

for batch in dataset_val.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

## Model Training

In [ ]:
model = getModel(inputs)
model.summary()

In [ ]:
history = model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val,
)

## Results

In [ ]:
visualize_loss(history)
visualize_accuracy(history)

In [ ]:
for x, y in dataset_val.take(5):
    show_result(x, y, future, model)

In [ ]:
# Now, I need to make a proper decision on how my hyperparameters are chosen. Speak to Tingting/Jon today if I have questions.